## 기상청 API 허브

DATA 수집 목표 : 2012 ~ (최소기간)

https://apihub.kma.go.kr/

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

import urllib
import urllib.request
from urllib.parse import unquote

import json

import requests
import pprint

from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm

### Text

In [23]:
year = [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
year2 = [2012, 2016, 2020]#, 2023]
# month = [1, 4, 7, 10]

In [ ]:

def short_term(year):
    url = f'https://apihub.kma.go.kr/api/typ01/url/fct_afs_dl.php?reg=&tmfc1={year}010100&tmfc2={year+1}010100&disp=1&help=0&authKey=XZ15B2mZRGqdeQdpmURqUg'
    response = requests.get(url)
    text = response.text.split('\n')
    return text
    

In [10]:
def short_term_half(year, a, b):
    
    url = f'https://apihub.kma.go.kr/api/typ01/url/fct_afs_dl.php?reg=&tmfc1={year}{a}0100&tmfc2={year}{b}0100&disp=1&help=0&authKey=yEtC0d2xQmKLQtHdsWJi5g'
    response = requests.get(url)
    text = response.text.split('\n')
    return text
    

In [73]:
url = 'https://apihub.kma.go.kr/api/typ01/url/fct_afs_dl.php?reg=&tmfc1=2023010100&tmfc2=2023031300&disp=1&help=0&authKey=yEtC0d2xQmKLQtHdsWJi5g'
response = requests.get(url)
text = response.text.split('\n')

SyntaxError: 'return' outside function (1628335635.py, line 4)

In [74]:
text

['#START7777',
 '# REG_ID TM_FC        TM_EF        MOD NE STN C MAN_ID       MAN_FC     W1 T W2  TA  ST SKY  PREP WF',
 '11A00101,202301010500,202301010000,A02,0,109,2,she,강혜미,N,1,NE,-99,0,DB01,0,맑음,=',
 '11A00101,202301010500,202301011200,A02,1,109,2,she,강혜미,NW,1,N,2,10,DB03,0,구름많음,=',
 '11A00101,202301010500,202301020000,A02,2,109,2,she,강혜미,NW,1,N,-4,10,DB03,0,구름많음,=',
 '11A00101,202301010500,202301021200,A02,3,109,2,she,강혜미,NW,1,N,0,10,DB01,0,맑음,=',
 '11A00101,202301010500,202301030000,A02,4,109,2,she,강혜미,NW,1,N,-3,10,DB03,0,구름많음,=',
 '11A00101,202301010500,202301031200,A02,5,109,2,she,강혜미,NW,1,N,2,10,DB01,0,맑음,=',
 '11A00101,202301010500,202301040000,A02,6,109,2,she,강혜미,NE,1,E,-2,20,DB03,0,구름많음,=',
 '11A00101,202301010500,202301041200,A02,7,109,2,she,강혜미,N,1,NE,2,10,DB03,0,구름많음,=',
 '11B10101,202301010500,202301010000,A02,0,109,2,she,강혜미,NW,1,N,-99,0,DB01,0,맑음,=',
 '11B10101,202301010500,202301011200,A02,1,109,2,she,강혜미,NW,1,N,2,10,DB03,0,구름많음,=',
 '11B10101,202301010500,202301020

In [ ]:
lst = []

for i in tqdm(year2):
    lst.append(short_term(i))

In [ ]:
# 2012, 2016, 2020 : 기간 초과

In [5]:
year_test = [2012]

In [24]:
lst_half = []

for i in tqdm(year2):
    lst_half.append(short_term_half(i, '01', '06'))
    lst_half.append(short_term_half(i, '07', '12'))
    

100%|█████████████████████████████████████████████████████████████████| 3/3 [06:07<00:00, 122.39s/it]


In [42]:
for i in range(6):
    print(lst_half[i][:50])

['#START7777', '# REG_ID TM_FC        TM_EF        MOD NE STN C MAN_ID       MAN_FC     W1 T W2  TA  ST SKY  PREP WF', '11A00101,201201010500,201201010000,A02,0,112,2,*,남영만,NW,1,N,-99,20,DB03,0,구름많음,=', '11A00101,201201010500,201201011200,A02,1,112,2,*,남영만,NW,1,N,1,10,DB02,0,구름조금,=', '11A00101,201201010500,201201020000,A02,2,112,2,*,남영만,NW,1,N,-4,10,DB02,0,구름조금,=', '11A00101,201201010500,201201021200,A02,3,112,2,*,남영만,NW,1,N,1,20,DB03,0,구름많음,=', '11B10101,201201010500,201201010000,A02,0,108,2,*,이미선,W,1,NW,-99,20,DB03,0,구름많음,=', '11B10101,201201010500,201201011200,A02,1,108,2,*,이미선,NW,1,N,1,20,DB03,0,구름많음,=', '11B10101,201201010500,201201020000,A02,2,108,2,*,이미선,W,1,NW,-7,0,DB01,0,맑음,=', '11B10101,201201010500,201201021200,A02,3,108,2,*,이미선,W,1,NW,0,10,DB02,0,구름조금,=', '11B10102,201201010500,201201010000,A02,0,109,2,*,최주권,W,1,NW,-99,20,DB03,0,구름많음,=', '11B10102,201201010500,201201011200,A02,1,109,2,*,최주권,NW,1,N,1,20,DB03,0,구름많음,=', '11B10102,201201010500,201201020000,A02,2,109,2,*,최주권,W,

In [ ]:
len(lst_half)

In [ ]:
lst_half[4][:-5]

In [ ]:
col_name = 'REG_ID TM_FC TM_EF        MOD NE STN C MAN_ID       MAN_FC     W1 T W2  TA  ST SKY  PREP WF'.split()
len(col_name)

In [ ]:
for i in range(8):
    print(lst[i])

tmfc1=2022060100&tmfc2=2023031300

In [ ]:
text

In [75]:
text[-5:]

['21F20804,202303121700,202303151200,A02,6,156,2,hlkim,김혜림,NW,1,N,18,30,DB04,0,흐림,=',
 '21F20804,202303121700,202303160000,A02,7,156,2,hlkim,김혜림,N,1,NE,4,30,DB04,0,흐림,=',
 '21F20804,202303121700,202303161200,A02,8,156,2,hlkim,김혜림,N,1,NE,15,30,DB04,0,흐림,=',
 '#7777END',
 '']

In [47]:
lst_half[1][1]

'# REG_ID TM_FC        TM_EF        MOD NE STN C MAN_ID       MAN_FC     W1 T W2  TA  ST SKY  PREP WF'

In [48]:
# columns 추출
col_name = lst_half[1][1].split()
col_name = col_name[1:]
print(col_name)
print(len(col_name))

['REG_ID', 'TM_FC', 'TM_EF', 'MOD', 'NE', 'STN', 'C', 'MAN_ID', 'MAN_FC', 'W1', 'T', 'W2', 'TA', 'ST', 'SKY', 'PREP', 'WF']
17


In [ ]:
# col_name = [i.strip() for i in col_name]

In [ ]:
# col_name[0] = 'TM_FC'

In [76]:
col_name

['REG_ID',
 'TM_FC',
 'TM_EF',
 'MOD',
 'NE',
 'STN',
 'C',
 'MAN_ID',
 'MAN_FC',
 'W1',
 'T',
 'W2',
 'TA',
 'ST',
 'SKY',
 'PREP',
 'WF']

In [ ]:
# 컬럼 시작 숫자 설정
text[-2]

In [57]:
for i in range(6):
    print(lst_half[i][-2:])

['#7777END', '']
['#7777END', '']
['#7777END', '']
['#7777END', '']
['#7777END', '']
['#7777END', '']


In [79]:
text[1]

'# REG_ID TM_FC        TM_EF        MOD NE STN C MAN_ID       MAN_FC     W1 T W2  TA  ST SKY  PREP WF'

In [84]:
df_2023 = pd.DataFrame(text[2:-2])[0].str.split(',', expand=True)
df_2023.drop(columns=17, inplace=True, axis=1)
df_2023.columns = col_name
df_2023

,REG_ID,TM_FC,TM_EF,MOD,NE,STN,C,MAN_ID,MAN_FC,W1,T,W2,TA,ST,SKY,PREP,WF
0,11A00101,202301010500,202301010000,A02,0,109,2,she,강혜미,N,1,NE,-99,0,DB01,0,맑음
1,11A00101,202301010500,202301011200,A02,1,109,2,she,강혜미,NW,1,N,2,10,DB03,0,구름많음
2,11A00101,202301010500,202301020000,A02,2,109,2,she,강혜미,NW,1,N,-4,10,DB03,0,구름많음
3,11A00101,202301010500,202301021200,A02,3,109,2,she,강혜미,NW,1,N,0,10,DB01,0,맑음
4,11A00101,202301010500,202301030000,A02,4,109,2,she,강혜미,NW,1,N,-3,10,DB03,0,구름많음
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
342499,21F20804,202303121700,202303141200,A02,4,156,2,hlkim,김혜림,SE,1,S,16,0,DB01,0,맑음
342500,21F20804,202303121700,202303150000,A02,5,156,2,hlkim,김혜림,S,1,SW,6,0,DB01,0,맑음
342501,21F20804,202303121700,202303151200,A02,6,156,2,hlkim,김혜림,NW,1,N,18,30,DB04,0,흐림
342502,21F20804,202303121700,202303160000,A02,7,156,2,hlkim,김혜림,N,1,NE,4,30,DB04,0,흐림


In [ ]:
df[9].value_counts()

In [ ]:
'''
# 띄어쓰기에 따라 나눠진 컬럼 합치기

con_cols = [18, 19, 20, 21]

df['con_cols'] = df[con_cols].apply(lambda row : ' '.join(row.values.astype(str)), axis=1)
df.drop(columns = con_cols, inplace = True)
'''

In [ ]:
df = df.iloc[:,:11]

In [ ]:
df.drop(columns = 11, inplace=True)

In [ ]:
# df_2013, df_2014, df_2015, df_2017, df_2018, df_2019, df_2021, df_2022

df1 = pd.concat([df_2013, df_2014, df_2015, df_2017, df_2018, df_2019, df_2021, df_2022])
df1.drop(columns = [17], inplace=True)
df1.columns = col_name
df1

In [60]:
df2 = pd.concat([df_2012, df_2016, df_2020])
df2.drop(columns = [17], inplace=True)
df2.columns = col_name
df2

,REG_ID,TM_FC,TM_EF,MOD,NE,STN,C,MAN_ID,MAN_FC,W1,T,W2,TA,ST,SKY,PREP,WF
0,11A00101,201201010500,201201011200,A02,1,112,2,*,남영만,NW,1,N,1,10,DB02,0,구름조금
1,11A00101,201201010500,201201020000,A02,2,112,2,*,남영만,NW,1,N,-4,10,DB02,0,구름조금
2,11A00101,201201010500,201201021200,A02,3,112,2,*,남영만,NW,1,N,1,20,DB03,0,구름많음
3,11B10101,201201010500,201201010000,A02,0,108,2,*,이미선,W,1,NW,-99,20,DB03,0,구름많음
4,11B10101,201201010500,201201011200,A02,1,108,2,*,이미선,NW,1,N,1,20,DB03,0,구름많음
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491258,21F20803,201205312300,201206020000,A02,3,156,2,*,주형돈,N,1,NE,17,20,DB03,0,구름많음
491259,21F20804,201205312300,201205311200,A02,0,156,2,*,주형돈,N,1,NE,-99,10,DB02,0,구름조금
491260,21F20804,201205312300,201206010000,A02,1,156,2,*,주형돈,NW,1,N,15,10,DB02,0,구름조금
491261,21F20804,201205312300,201206011200,A02,2,156,2,*,주형돈,NW,1,N,26,20,DB03,0,구름많음


In [85]:
# 필요없는 COLUMN 제거 구역코드(REG_ID), 예보관ID(MAN_ID), 예보관명(MAN_FC)
# 구간(MOD) 12, 24 시간
# SKY + PREP 한글화 = WF

df_2023.drop(columns = ['REG_ID', 'MAN_ID', 'MAN_FC'], inplace=True)

In [86]:
df_2023

,TM_FC,TM_EF,MOD,NE,STN,C,W1,T,W2,TA,ST,SKY,PREP,WF
0,202301010500,202301010000,A02,0,109,2,N,1,NE,-99,0,DB01,0,맑음
1,202301010500,202301011200,A02,1,109,2,NW,1,N,2,10,DB03,0,구름많음
2,202301010500,202301020000,A02,2,109,2,NW,1,N,-4,10,DB03,0,구름많음
3,202301010500,202301021200,A02,3,109,2,NW,1,N,0,10,DB01,0,맑음
4,202301010500,202301030000,A02,4,109,2,NW,1,N,-3,10,DB03,0,구름많음
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
342499,202303121700,202303141200,A02,4,156,2,SE,1,S,16,0,DB01,0,맑음
342500,202303121700,202303150000,A02,5,156,2,S,1,SW,6,0,DB01,0,맑음
342501,202303121700,202303151200,A02,6,156,2,NW,1,N,18,30,DB04,0,흐림
342502,202303121700,202303160000,A02,7,156,2,N,1,NE,4,30,DB04,0,흐림


In [62]:
df2.to_csv('단기육상예보_2차.csv')

### XML

In [ ]:
url = 'https://apihub.kma.go.kr/api/typ02/openApi/SfcYearlyInfoService/getTyphoonList?pageNo=1&numOfRows=10&dataType=XML&year=2022&authKey=_v7wW5IUQqu-8FuSFNKrLA'

response = requests.get(url)

# xml 내용
content = response.text

# 깔끔한 출력 위한 코드
pp = pprint.PrettyPrinter(indent=4)
print(pp.pprint(content))

In [ ]:
from os import name
import xml.etree.ElementTree as et
import bs4
from lxml import html
from urllib.parse import urlencode, quote_plus, unquote

#bs4 사용하여 item 태그 분리
xml_obj = bs4.BeautifulSoup(content,'lxml-xml')
rows = xml_obj.findAll('info') # item 으로 구분되어 있는 경우도 있음
print(rows)

In [ ]:
# 각 행의 컬럼, 이름, 값을 가지는 리스트 만들기
row_list = [] # 행값
name_list = [] # 열이름값
value_list = [] #데이터값

# xml 안의 데이터 수집
for i in range(0, len(rows)):
    columns = rows[i].find_all()
    #첫째 행 데이터 수집
    for j in range(0,len(columns)):
        if i ==0:
            # 컬럼 이름 값 저장
            name_list.append(columns[j].name)
        # 컬럼의 각 데이터 값 저장
        value_list.append(columns[j].text)
    # 각 행의 value값 전체 저장
    row_list.append(value_list)
    # 데이터 리스트 값 초기화
    value_list=[]

In [ ]:
#xml값 DataFrame으로 만들기
xml_df = pd.DataFrame(row_list, columns=name_list)
print(xml_df.head())

In [ ]:
xml_df

In [ ]:
xml_df.to_csv('우리나라영향태풍_2021.csv')

# Data

In [66]:
df1 = pd.read_csv('G:/내 드라이브/취업/Contest/2307_PublicData/data/단기육상예보_1차.csv', index_col=0)
df2 = pd.read_csv('G:/내 드라이브/취업/Contest/2307_PublicData/data/단기육상예보_2차.csv', index_col=0)
df3 = pd.read_csv('G:/내 드라이브/취업/Contest/2307_PublicData/data/단기육상예보_3차.csv', index_col=0)

In [94]:
df2

,TM_FC,TM_EF,MOD,NE,STN,C,W1,T,W2,TA,ST,SKY,PREP,WF
0,201201010500,201201011200,A02,1,112,2,NW,1,N,1,10,DB02,0,구름조금
1,201201010500,201201020000,A02,2,112,2,NW,1,N,-4,10,DB02,0,구름조금
2,201201010500,201201021200,A02,3,112,2,NW,1,N,1,20,DB03,0,구름많음
3,201201010500,201201010000,A02,0,108,2,W,1,NW,-99,20,DB03,0,구름많음
4,201201010500,201201011200,A02,1,108,2,NW,1,N,1,20,DB03,0,구름많음
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491258,201205312300,201206020000,A02,3,156,2,N,1,NE,17,20,DB03,0,구름많음
491259,201205312300,201205311200,A02,0,156,2,N,1,NE,-99,10,DB02,0,구름조금
491260,201205312300,201206010000,A02,1,156,2,NW,1,N,15,10,DB02,0,구름조금
491261,201205312300,201206011200,A02,2,156,2,NW,1,N,26,20,DB03,0,구름많음


In [95]:
df_2023

,TM_FC,TM_EF,MOD,NE,STN,C,W1,T,W2,TA,ST,SKY,PREP,WF
0,202301010500,202301010000,A02,0,109,2,N,1,NE,-99,0,DB01,0,맑음
1,202301010500,202301011200,A02,1,109,2,NW,1,N,2,10,DB03,0,구름많음
2,202301010500,202301020000,A02,2,109,2,NW,1,N,-4,10,DB03,0,구름많음
3,202301010500,202301021200,A02,3,109,2,NW,1,N,0,10,DB01,0,맑음
4,202301010500,202301030000,A02,4,109,2,NW,1,N,-3,10,DB03,0,구름많음
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
342499,202303121700,202303141200,A02,4,156,2,SE,1,S,16,0,DB01,0,맑음
342500,202303121700,202303150000,A02,5,156,2,S,1,SW,6,0,DB01,0,맑음
342501,202303121700,202303151200,A02,6,156,2,NW,1,N,18,30,DB04,0,흐림
342502,202303121700,202303160000,A02,7,156,2,N,1,NE,4,30,DB04,0,흐림


In [119]:
df = pd.concat([df1, df2, df_2023]).reset_index()
df

,index,TM_FC,TM_EF,MOD,NE,STN,C,W1,T,W2,TA,ST,SKY,PREP,WF
0,0,201301010500,201301011200,A02,1,112,2,NW,1,N,1,60,DB04,3,흐리고 한때 눈
1,1,201301010500,201301020000,A02,2,112,2,NW,1,N,-10,70,DB04,3,흐리고 눈
2,2,201301010500,201301021200,A02,3,112,2,NW,1,N,-9,60,DB04,3,흐리고 한때 눈
3,3,201301010500,201301010000,A02,0,108,2,E,1,SE,-99,60,DB04,3,흐리고 한때 눈
4,4,201301010500,201301011200,A02,1,108,2,SW,1,W,-1,60,DB04,3,흐리고 가끔 눈
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11530274,342499,202303121700,202303141200,A02,4,156,2,SE,1,S,16,0,DB01,0,맑음
11530275,342500,202303121700,202303150000,A02,5,156,2,S,1,SW,6,0,DB01,0,맑음
11530276,342501,202303121700,202303151200,A02,6,156,2,NW,1,N,18,30,DB04,0,흐림
11530277,342502,202303121700,202303160000,A02,7,156,2,N,1,NE,4,30,DB04,0,흐림


In [120]:
df.drop(columns='index', inplace=True)

In [121]:
df

,TM_FC,TM_EF,MOD,NE,STN,C,W1,T,W2,TA,ST,SKY,PREP,WF
0,201301010500,201301011200,A02,1,112,2,NW,1,N,1,60,DB04,3,흐리고 한때 눈
1,201301010500,201301020000,A02,2,112,2,NW,1,N,-10,70,DB04,3,흐리고 눈
2,201301010500,201301021200,A02,3,112,2,NW,1,N,-9,60,DB04,3,흐리고 한때 눈
3,201301010500,201301010000,A02,0,108,2,E,1,SE,-99,60,DB04,3,흐리고 한때 눈
4,201301010500,201301011200,A02,1,108,2,SW,1,W,-1,60,DB04,3,흐리고 가끔 눈
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11530274,202303121700,202303141200,A02,4,156,2,SE,1,S,16,0,DB01,0,맑음
11530275,202303121700,202303150000,A02,5,156,2,S,1,SW,6,0,DB01,0,맑음
11530276,202303121700,202303151200,A02,6,156,2,NW,1,N,18,30,DB04,0,흐림
11530277,202303121700,202303160000,A02,7,156,2,N,1,NE,4,30,DB04,0,흐림


In [122]:
df = df.sort_values(by='TM_FC', ascending=True)

In [117]:
df_2023['TM_FC'] = df_2023['TM_FC'].astype('int64')

In [124]:
df.reset_index(drop=True)

,TM_FC,TM_EF,MOD,NE,STN,C,W1,T,W2,TA,ST,SKY,PREP,WF
0,201201010500,201201011200,A02,1,112,2,NW,1,N,1,10,DB02,0,구름조금
1,201201010500,201201011200,A02,1,143,2,W,1,NW,3,10,DB02,0,구름조금
2,201201010500,201201020000,A02,2,143,2,W,1,NW,-11,0,DB01,0,맑음
3,201201010500,201201021200,A02,3,143,2,W,1,NW,3,10,DB02,0,구름조금
4,201201010500,201201010000,A02,0,143,2,W,1,NW,-99,20,DB03,0,구름많음
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11530274,202303121700,202303131200,A02,2,105,2,W,1,NW,6,20,DB03,0,구름많음
11530275,202303121700,202303130000,A02,1,105,2,W,1,NW,-7,0,DB01,0,맑음
11530276,202303121700,202303121200,A02,0,105,2,W,1,NW,-99,60,DB04,2,흐리고 한때 비/눈
11530277,202303121700,202303161200,A02,8,105,2,W,1,NW,13,30,DB04,0,흐림


In [28]:
df2 = pd.read_csv('G:/내 드라이브/취업/Contest/2307_PublicData/data/적설관측자료(2023).csv')
df2

,Unnamed: 0,YYMMDDHHMI,STN,STN.1,LON,LAT,STN.2,SD_TOT,SD_DAY,SD_HR3,SD_D01,SD_H03,SD_H24,SD_H01
0,0,202301010000,90,속초,128.56473,38.25085,000-----,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
1,1,202301010100,90,속초,128.56473,38.25085,000-----,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
2,2,202301010200,90,속초,128.56473,38.25085,000-----,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
3,3,202301010300,90,속초,128.56473,38.25085,000-----,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
4,4,202301010400,90,속초,128.56473,38.25085,000-----,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
787705,787705,202303122000,5343,포항,129.37963,36.03259,000-----,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
787706,787706,202303122100,5343,포항,129.37963,36.03259,000-----,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
787707,787707,202303122200,5343,포항,129.37963,36.03259,000-----,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
787708,787708,202303122300,5343,포항,129.37963,36.03259,000-----,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0


In [37]:
for i in df2['STN.1'].value_counts().index : print(i)

포항
속초
심원
동향
진안주천
함평
어리목
서광
백야
도양
성전
산이
운남
화순
줄포
태인
김제
진봉
덕유산
완주
함라
보성
영암
장성
유수암
추자도
조선대
금일
복흥
강진면
선유도
신덕
영양
평해
하태도
초도
옥과
황전
표선
압해도
풍암
도화
돌산
북일
시종
과기원
한림
유치
관산
현산
월야
전남도청
학산
다도
염산
곡성
여수산단
보길도
의신
옥산
양양
현내
면온
만리포
대천항
볼음도
자월도
외촌
양지
부평
춘장대
서부
연무
세천
오월드
가덕
청천
덕산
신평
계룡
양화
아산
정안
유구
송도
노은
영흥도
청호
하의도
상하
이양
나주
구례
광산
담양
변산
진안
익산
무주
무안
해제
신기
원효봉
백학
정산
풍산
광주남구
석곡
평화
강릉왕산
강릉성산
달방댐
위성센터
사북
진영
온정
청하
예산
함안
고성
사천
삼가
서하
상주면
백천
진해
삼천포
화개
양산상북
사상
도천
시천
울기
두서
의왕이동
군포금정
애월
성산수산
제주가시리
군산산단
태풍센터
새별오름
상무대
새만금
창녕
단장
죽장
기장
하빈
옥포
낙천
강정
한라산남벽
마량
장목
정자
명사
북상
대병
길곡
부산남구
북구
동래
금정구
부산진
신포
청덕
수곡
금남
하동
개천
송백
진북
문막
삼척
설악산
동송
화서
은척
석포
기계
외동
경산
신녕
칠곡
가산
군위
김천
하회
예안
마성
수비
구룡포
예천
부석
청도
고령
황성
성주
대덕
의흥
선산
봉화읍
동로
길안
가파도
지리산
사제비
삼각봉
한라생태숲
오등
완산
송당
월정
신암
토함산
완도읍
팔공산
화북
죽변
소곡
감포
지보
청도금천
소보
대구서구
대구북구
영덕읍
금강송
산내
세종전의
태안
북춘천
청송군
영주
봉화
진도군
광양시
함양군
의령군
고흥
해남
장흥
강진군
보성군
양산시
북창원
순창군
김해시
영광군
고창군
장수
남원
정읍
임실
부안
세종
금산
부여
문경
영덕
산천단
의성
청주금천
용문산
송계
마현
상서
원통
향로봉
천부
용평
구이
주왕산
판문점
소리도
가거도
말도
북부산
남해
거제
산청
밀양
합천
거창
경주시
영천
구미
보령
천안
덕적북리
보은
대구
군산
상주
안동

In [126]:
df.to_csv('단기육상예보.csv')

OSError: [Errno 28] No space left on device

In [128]:
df_2023.to_csv('단기육상예보_3차.csv')